In [1]:
import pandas as pd
import numpy as np
import spacy
import art.config
art.config.ART_NUMPY_DTYPE = 'str' # override dtype to str instead of float

from art.estimators.classification import BlackBoxClassifier
from art.attacks.extraction import KnockoffNets
from spacy_clf import SpacyClassifier

In [2]:
texts = pd.read_csv('res/fake-news/train.csv')['text'].to_numpy()

In [3]:
blackbox_classifier = SpacyClassifier(model = spacy.load('blackbox-models/fake-news/'))

In [4]:
knockoff = KnockoffNets(classifier = blackbox_classifier, batch_size_fit=32, batch_size_query=32, nb_stolen=100, sampling_strategy='random')

In [5]:
np.random.seed(23435432)
thieved_classifier = SpacyClassifier(model = spacy.load("en_core_web_sm"), target_classes=['False', 'True'])
knockoff.extract(x = texts, thieved_classifier=thieved_classifier)

SpacyClassifier(model=<spacy.lang.en.English object at 0x0000019E9EBDF7F0>, clip_values=None, preprocessing_defences=None, postprocessing_defences=None, preprocessing=(0, 1), input_shape=(1,), classes=['False', 'True'], nb_classes=2, architecture=simple_cnn, exclusive_classes=True)

In [6]:
blackbox_classifier.predict(texts[-40:])

array([[1.00000000e+00, 1.51131019e-08],
       [9.99956489e-01, 4.35040492e-05],
       [9.99998927e-01, 1.09775499e-06],
       [1.00000000e+00, 2.86335378e-09],
       [1.00000000e+00, 2.04748769e-08],
       [9.99999881e-01, 7.37204786e-08],
       [9.99999881e-01, 1.77476437e-07],
       [1.00000000e+00, 1.79037976e-08],
       [9.99999881e-01, 1.32739643e-07],
       [1.00000000e+00, 1.35752840e-08],
       [9.99999881e-01, 1.17084134e-07],
       [9.99999762e-01, 1.94951312e-07],
       [1.00000000e+00, 2.45807730e-08],
       [9.99999881e-01, 8.12786638e-08],
       [9.99949217e-01, 5.07527257e-05],
       [9.99999881e-01, 1.53873671e-07],
       [9.99999762e-01, 2.06596113e-07],
       [1.00000000e+00, 4.81961493e-09],
       [1.00000000e+00, 7.53924301e-09],
       [9.99998093e-01, 1.87481555e-06],
       [9.99999881e-01, 8.55446629e-08],
       [9.99999881e-01, 1.78417963e-07],
       [9.99999523e-01, 4.69952425e-07],
       [9.99999881e-01, 7.61233210e-08],
       [9.999998

In [7]:
thieved_classifier.predict(texts[:40])

array([[3.32257259e-05, 9.99966741e-01],
       [2.31558647e-06, 9.99997735e-01],
       [2.46292693e-05, 9.99975324e-01],
       [2.65831045e-06, 9.99997377e-01],
       [2.68317708e-05, 9.99973178e-01],
       [1.05078077e-04, 9.99894857e-01],
       [8.25811730e-06, 9.99991775e-01],
       [3.04023306e-05, 9.99969602e-01],
       [7.66028916e-06, 9.99992371e-01],
       [7.82315692e-05, 9.99921799e-01],
       [5.73770149e-06, 9.99994278e-01],
       [1.81871019e-05, 9.99981761e-01],
       [3.78332561e-06, 9.99996185e-01],
       [1.40686316e-05, 9.99985933e-01],
       [1.41176897e-05, 9.99985933e-01],
       [6.37812900e-06, 9.99993563e-01],
       [2.25230679e-06, 9.99997735e-01],
       [9.24116175e-05, 9.99907613e-01],
       [5.99971690e-06, 9.99994040e-01],
       [2.06773300e-04, 9.99793231e-01],
       [7.78324284e-06, 9.99992251e-01],
       [2.06956483e-05, 9.99979258e-01],
       [1.05684248e-05, 9.99989390e-01],
       [3.38012910e-06, 9.99996662e-01],
       [2.891431